In [1]:
import os
import cv2
import face_recognition

# 画像のダウンロード

In [2]:
urls = [
    'https://www.kantei.go.jp/jp/content/20210730_corona01.jpg',
    'https://www.cnn.co.jp/storage/2020/04/13/347e81ac7c26f732f5fe7f9f21cda3a7/t/768/432/d/joe-biden-donald-trump-split-file-super-16-9.jpg',
    'https://www.kensyu-center.jp/case/wp/wp-content/uploads/2016/02/08-04_33000000829_R.jpg',
    'http://kf-camera.com/wp-content/uploads/2016/01/20160112_8.jpg',
    'https://pictkan.com/uploads/converted/15/06/15/2380770682-cycling-664753-EkN-1280x853-MM-100.jpg']

In [3]:
import subprocess

# ホームディレクトリの下に'img'ディレクトリ作成
imgdir = os.path.join(os.path.expanduser('~'), 'img')
os.makedirs(imgdir, exist_ok=True)

for url in urls:
    fname = url[url.rfind('/')+1:]
    imgpath = os.path.join(imgdir, fname)
    
    # 画像ファイルがなければダウンロードする。
    if not os.path.isfile(imgpath):
        result = subprocess.run(['curl', '-O', url], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, cwd=imgdir)

# 画像のダウンサンプリング

In [4]:
filelist = [ 'sample1.jpg',  'sample2.jpg', 'sample3.jpg', 
            '20210730_corona01.jpg', 'joe-biden-donald-trump-split-file-super-16-9.jpg',
            '08-04_33000000829_R.jpg', '20160112_8.jpg', 
            '2380770682-cycling-664753-EkN-1280x853-MM-100.jpg']
imgdir = os.path.join(os.path.expanduser('~'), 'img')
filepaths = [os.path.join(imgdir, file) for file in filelist]

In [5]:
for filepath in filepaths:
    img = cv2.imread(filepath)

    w, h = img.shape[:2]
    scale = min(800/w, 800/h)
    img = cv2.resize(img, dsize=None, fx=scale, fy=scale)
    
    cv2.imwrite(filepath, img)

# 検出領域を可視化し保存する関数

In [6]:
def save_face_located_image(filepath, face_locations, prefix):
    # 保存用の画像データを準備
    save_img = cv2.imread(filepath)
    
    # 検出した範囲を保存用画像に短形表示
    for face_location in face_locations:
        y1, x2, y2, x1 = face_location
        cv2.rectangle(save_img, (x1, y1), (x2, y2), color=(0,0,255), thickness=4)
        
    # 検出領域を示した画像を保存
    filename = os.path.basename(filepath)
    filedir = os.path.dirname(filepath)
    save_filename = prefix + '_' + filename
    save_imgpath = os.path.join(filedir, save_filename)
    cv2.imwrite(save_imgpath, save_img)

# HOG特徴量ベースの顔検出

In [7]:
for filepath in filepaths:
    print("------------------------------")
    print(f"Processing file: {os.path.basename(filepath)}")
    
    # 画像読み込み
    image = face_recognition.load_image_file(filepath)
    
    # HOGベースモデルの顔検出実行
    face_locations = face_recognition.face_locations(image)
    
    # 検出した領域を表示
    print("I found {} face(s) in this photograph.".format(len(face_locations)))
    for face_location in face_locations:
        top, right, bottom, left = face_location
        print("A face is located at pixel location Top: {}, Left: {}, Bottom: {}, Right: {}".format(top, left, bottom, right))
        
    # 顔検出した領域を短形表示した画像を保存
    save_face_located_image(filepath, face_locations, 'hog')

------------------------------
Processing file: sample1.jpg
I found 1 face(s) in this photograph.
A face is located at pixel location Top: 160, Left: 97, Bottom: 345, Right: 283
------------------------------
Processing file: sample2.jpg
I found 4 face(s) in this photograph.
A face is located at pixel location Top: 59, Left: 266, Bottom: 122, Right: 328
A face is located at pixel location Top: 184, Left: 128, Bottom: 246, Right: 190
A face is located at pixel location Top: 73, Left: 411, Bottom: 135, Right: 474
A face is located at pixel location Top: 191, Left: 557, Bottom: 253, Right: 619
------------------------------
Processing file: sample3.jpg
I found 1 face(s) in this photograph.
A face is located at pixel location Top: 113, Left: 377, Bottom: 164, Right: 429
------------------------------
Processing file: 20210730_corona01.jpg
I found 1 face(s) in this photograph.
A face is located at pixel location Top: 139, Left: 339, Bottom: 268, Right: 468
------------------------------
Pro

# CNNベースの顔検出

In [8]:
for filepath in filepaths:
    print("------------------------------")
    print(f"Processing file: {os.path.basename(filepath)}")
    
    # 画像読み込み
    image = face_recognition.load_image_file(filepath)
    
    # CNNベースモデルの顔検出実行
    face_locations = face_recognition.face_locations(image, number_of_times_to_upsample=1, model="cnn")
    
    # 検出した領域を表示
    print("I found {} face(s) in this photograph.".format(len(face_locations)))
    for face_location in face_locations:
        top, right, bottom, left = face_location
        print("A face is located at pixel location Top: {}, Left: {}, Bottom: {}, Right: {}".format(top, left, bottom, right))
        
    # 顔検出した領域を短形表示した画像を保存
    save_face_located_image(filepath, face_locations, 'cnn')

------------------------------
Processing file: sample1.jpg
I found 1 face(s) in this photograph.
A face is located at pixel location Top: 148, Left: 75, Bottom: 352, Right: 279
------------------------------
Processing file: sample2.jpg
I found 5 face(s) in this photograph.
A face is located at pixel location Top: 68, Left: 422, Bottom: 125, Right: 479
A face is located at pixel location Top: 189, Left: 569, Bottom: 257, Right: 637
A face is located at pixel location Top: 178, Left: 128, Bottom: 235, Right: 185
A face is located at pixel location Top: 63, Left: 278, Bottom: 119, Right: 335
A face is located at pixel location Top: 114, Left: 661, Bottom: 212, Right: 759
------------------------------
Processing file: sample3.jpg
I found 4 face(s) in this photograph.
A face is located at pixel location Top: 91, Left: 286, Bottom: 159, Right: 354
A face is located at pixel location Top: 113, Left: 457, Bottom: 160, Right: 504
A face is located at pixel location Top: 113, Left: 375, Botto